In [1]:
#import necessary libraries
import numpy as np
import json
import random
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf

In [ ]:
# Load the intents from the JSON file
with open('C:\\Users\\Dishant\\Desktop\\Mlsccoherence_teamRadhey\\cva\\intents.json', 'r') as file:
    intents = json.load(file)['intents']

In [ ]:
!pip install nltk

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
import nltk
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Initialize the lemmatizer and tokenizer
lemmatizer = WordNetLemmatizer()
tokenizer = word_tokenize
# Create a list of all words in the intents, and a list of all intents
words = []
classes = []
documents = []
for intent in intents:
    for pattern in intent['patterns']:
        # Tokenize and lemmatize each word in the pattern
        words_in_pattern = tokenizer(pattern.lower())
        words_in_pattern = [lemmatizer.lemmatize(word) for word in words_in_pattern]
        # Add the words to the list of all words
        words.extend(words_in_pattern)
        # Add the pattern and intent to the list of all documents
        documents.append((words_in_pattern, intent['tag']))
        # Add the intent to the list of all intents
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# Remove duplicates and sort the words and classes
words = sorted(list(set(words)))
classes = sorted(classes)

In [ ]:
# Create training data as a bag of words
training_data = []
for document in documents:
    bag = []
    # Create a bag of words for each document
    for word in words:
        bag.append(1) if word in document[0] else bag.append(0)
    # Append the bag of words and the intent tag to the training data
    output_row = [0] * len(classes)
    output_row[classes.index(document[1])] = 1
    training_data.append([bag, output_row])

In [ ]:
# Shuffle the training data and split it into input and output lists
random.shuffle(training_data)
training_data = np.array(training_data, dtype=object)
train_x = list(training_data[:, 0])
train_y = list(training_data[:, 1])

In [ ]:

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(np.array(train_x), np.array(train_y), epochs=250, batch_size=5)


Epoch 1/250
26/26 [==============================] - 1s 3ms/step - loss: 3.7305 - accuracy: 0.0315
Epoch 2/250
26/26 [==============================] - 0s 3ms/step - loss: 3.6434 - accuracy: 0.0236
Epoch 3/250
26/26 [==============================] - 0s 3ms/step - loss: 3.5422 - accuracy: 0.0866
Epoch 4/250
26/26 [==============================] - 0s 3ms/step - loss: 3.5405 - accuracy: 0.0787
Epoch 5/250
26/26 [==============================] - 0s 3ms/step - loss: 3.4764 - accuracy: 0.1260
Epoch 6/250
26/26 [==============================] - 0s 3ms/step - loss: 3.3929 - accuracy: 0.1575
Epoch 7/250
26/26 [==============================] - 0s 3ms/step - loss: 3.2578 - accuracy: 0.1969
Epoch 8/250
26/26 [==============================] - 0s 3ms/step - loss: 3.1516 - accuracy: 0.2126
Epoch 9/250
26/26 [==============================] - 0s 3ms/step - loss: 3.1287 - accuracy: 0.1969
Epoch 10/250
26/26 [==============================] - 0s 3ms/step - loss: 2.9666 - accuracy: 0.2598
Epoch 11/

In [ ]:
# Define a function to process user input and generate a response
def get_response(user_input):
    # Tokenize and lemmatize the user input
    words_in_input = tokenizer(user_input.lower())
    words_in_input = [lemmatizer.lemmatize(word) for word in words_in_input]

    # Create a bag of words for the user input
    bag = [0] * len(words)
    for word in words_in_input:
        for i, w in enumerate(words):
            if w == word:
                bag[i] = 1

    # Predict the intent of the user input using the trained model
    results = model.predict(np.array([bag]), verbose=0)[0]
    # Get the index of the highest probability result
    index = np.argmax(results)
    # Get the corresponding intent tag
    tag = classes[index]

    # If the probability of the predicted intent is below a certain threshold, return a default response
    if results[index] < 0.5:
        return "I'm sorry, I don't understand. Can you please rephrase?"

    print(tag)
    # Get a random response from the intent
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])

    return response

# Main loop to get user input and generate responses
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
            break
    response = get_response(user_input)
    print("CVA ::", response)


You: hi
greeting
CVA :: Hi there! What can I do for you?
You: I want to return my order
cancel_order
CVA :: We apologize for any inconvenience. Please provide your order number and we will cancel your order for you.
You: my order number is 12345
cancel_order
CVA :: We apologize for any inconvenience. Please provide your order number and we will cancel your order for you.
You: My order number is 123456
cancel_order
CVA :: We apologize for any inconvenience. Please provide your order number and we will cancel your order for you.
You: my order number is {{123}}
oder_number
CVA ::  Thank you for waiting, sir. I apologize for the inconvenience caused. I see that the delivery was not attempted again due to an issue with the address provided. However, I assure you that we will take the necessary steps to ensure that you receive your order as soon as possible.
You: quit


In [ ]:
import keras
print(keras.__version__)

2.15.0


In [ ]:
import pickle

In [ ]:
pickle.dump((model,lemmatizer , tokenizer,), open('model.pkl', 'wb'))

In [ ]:
pickled_model,pickle_lemmatizer,pickle_tokenizer = pickle.load(open('model.pkl', 'rb'))

In [ ]:
# Define a function to process user input and generate a response
def get_response(user_input):
    # Tokenize and lemmatize the user input
    words_in_input = pickle_tokenizer(user_input.lower())
    words_in_input = [pickle_lemmatizer.lemmatize(word) for word in words_in_input]

    # Create a bag of words for the user input
    bag = [0] * len(words)
    for word in words_in_input:
        for i, w in enumerate(words):
            if w == word:
                bag[i] = 1

    # Predict the intent of the user input using the trained model
    results = pickled_model.predict(np.array([bag]), verbose=0)[0]
    # Get the index of the highest probability result
    index = np.argmax(results)
    # Get the corresponding intent tag
    tag = classes[index]

    # If the probability of the predicted intent is below a certain threshold, return a default response
    if results[index] < 0.5:
        return "I'm sorry, I don't understand. Can you please rephrase?"

    print(tag)
    # Get a random response from the intent
    for intent in intents:
        if intent['tag'] == tag:
            response = random.choice(intent['responses'])

    return response

# Main loop to get user input and generate responses
while True:
    user_input = input("You :: ")
    if user_input.lower() == "quit":
            break
    response = get_response(user_input)
    print("CVA :: ", response)


You: hi
greeting
CVA :: Hi there! What can I do for you?
You: how are you
situation
CVA :: I'am fine,what about you?
You: I am also good
greetreply
CVA :: Good to know!
You: return order
CVA :: I'm sorry, I don't understand. Can you please rephrase?
You: return my order
cancel_order
CVA :: Sure, we can cancel your order. Please provide your order number and we will process the cancellation for you.
You: order number is 123
order_status
CVA :: Our records show that your order is currently in transit. Please provide your order number if you need more information.
You: what are payments method
situation
CVA :: I'am fine,what about you?
You: what are payments method?
payment_methods
CVA :: Yes, we accept PayPal as a payment method.
You: what is the payment process?
payment_process
CVA :: Sure, Once you have added the product to your cart, you can proceed to the checkout page. On the checkout page, you will see the payment options. Please select the payment option that suits you the best an

In [ ]:
import pandas as pd

# Read the Parquet file
df = pd.read_parquet('/content/train-00000-of-00001-a5a7c6e4bb30b016.parquet')

# Save as CSV
df.to_csv('cva_train_data.csv', index=False)

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/cva_train_data.csv")
df

,issue_area,issue_category,issue_sub_category,issue_category_sub_category,customer_sentiment,product_category,product_sub_category,issue_complexity,agent_experience_level,agent_experience_level_desc,conversation
0,Login and Account,Mobile Number and Email Verification,Verification requirement for mobile number or ...,Mobile Number and Email Verification -> Verifi...,neutral,Appliances,Oven Toaster Grills (OTG),medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
1,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Electronics,Computer Monitor,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox customer...
2,Cancellations and returns,Replacement and Return Process,Inability to click the 'Cancel' button,Replacement and Return Process -> Inability to...,neutral,Appliances,Juicer/Mixer/Grinder,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for calling BrownBox Customer...
3,Login and Account,Login Issues and Error Messages,Error message regarding exceeded attempts to e...,Login Issues and Error Messages -> Error messa...,neutral,Appliances,Water Purifier,less,inexperienced,"may struggle with ambiguous queries, rely on c...","Customer: Hi, I am facing an issue while loggi..."
4,Order,Order Delivery Issues,Delivery not attempted again,Order Delivery Issues -> Delivery not attempte...,negative,Electronics,Bp Monitor,medium,experienced,"confidently handles complex customer issues, e...",Agent: Thank you for contacting BrownBox custo...
...,...,...,...,...,...,...,...,...,...,...,...
995,Cancellations and returns,Return and Exchange,Package open or tampered on delivery,Return and Exchange -> Package open or tampere...,negative,Electronics,Mobile,medium,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
996,Cancellations and returns,Pickup and Shipping,Reasons for being asked to ship the item,Pickup and Shipping -> Reasons for being asked...,neutral,Men/Women/Kids,Backpack,medium,junior,"handles customer inquiries independently, poss...","Customer: Hi, I received an email from BrownBo..."
997,Warranty,Warranty Terms and Changes,Warranty mismatch between the website and the ...,Warranty Terms and Changes -> Warranty mismatc...,negative,Appliances,Water Purifier,less,junior,"handles customer inquiries independently, poss...",Agent: Thank you for calling BrownBox Customer...
998,Cancellations and returns,Return and Exchange,Checking the status of a refund,Return and Exchange -> Checking the status of ...,neutral,Appliances,Wet Grinder,medium,junior,"handles customer inquiries independently, poss...","Customer: Hi, I would like to check the status..."


In [ ]:
# Initialize the lemmatizer and tokenizer
lemmatizer = WordNetLemmatizer()
tokenizer = word_tokenize
# Create a list of all words in the intents, and a list of all intents
words = []
classes = []
documents = []
for intent in intents:
    for pattern in df['issue_category_sub_category']:
        # Tokenize and lemmatize each word in the pattern
        words_in_pattern = tokenizer(pattern.lower())
        words_in_pattern = [lemmatizer.lemmatize(word) for word in words_in_pattern]
        # Add the words to the list of all words
        words.extend(words_in_pattern)
        # Add the pattern and intent to the list of all documents
        documents.append((words_in_pattern, df['issue_area']))
        # Add the intent to the list of all intents
        if df['issue_area'] not in classes:
            classes.append(df['issue_area'])

In [ ]:
# Remove duplicates and sort the words and classes
words = sorted(list(set(words)))
classes = sorted(classes)

In [ ]:
# Create training data as a bag of words
training_data = []
for document in documents:
    bag = []
    # Create a bag of words for each document
    for word in words:
        bag.append(1) if word in document[0] else bag.append(0)
    # Append the bag of words and the intent tag to the training data
    output_row = [0] * len(classes)
    output_row[classes.index(document[1])] = 1
    training_data.append([bag, output_row])

In [ ]:
# Shuffle the training data and split it into input and output lists
random.shuffle(training_data)
training_data = np.array(training_data, dtype=object)
train_x = list(training_data[:, 0])
train_y = list(training_data[:, 1])

In [ ]:
# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(np.array(train_x), np.array(train_y), epochs=5, batch_size=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/dispatch.py:1260: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(5, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


7400/7400 [==============================] - 26s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 2/5
7400/7400 [==============================] - 23s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/5
7400/7400 [==============================] - 23s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/5
7400/7400 [==============================] - 23s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/5
7400/7400 [==============================] - 23s 3ms/step - loss: 0.0000e+00 - accuracy: 1.0000


In [ ]:
# Define a function to process user input and generate a response
def get_response(user_input):
    # Tokenize and lemmatize the user input
    words_in_input = tokenizer(user_input.lower())
    words_in_input = [lemmatizer.lemmatize(word) for word in words_in_input]

    # Create a bag of words for the user input
    bag = [0] * len(words)
    for word in words_in_input:
        for i, w in enumerate(words):
            if w == word:
                bag[i] = 1

    # Predict the intent of the user input using the trained model
    results = model.predict(np.array([bag]), verbose=0)[0]
    # Get the index of the highest probability result
    index = np.argmax(results)
    # Get the corresponding intent tag
    tag = classes[index]

    # If the probability of the predicted intent is below a certain threshold, return a default response
    if results[index] < 0.5:
        return "I'm sorry, I don't understand. Can you please rephrase?"
    print("tags :: ",tag[1])
    # Get a random response from the intent
    for intent in df:
        if intent['issue_category_sub_category'] == tag[1]:
            response = random.choice(intent['responses'])

    return response

# Main loop to get user input and generate responses
while True:
    user_input = input("You: ")
    if user_input.lower() == "quit":
            break
    response = get_response(user_input)
    print("CVA ::", response)


You: hello
tags ::  Cancellations and returns


TypeError: string indices must be integers

In [ ]:
for i in intents


[{'tag': 'greeting',
  'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
  'responses': ['Hello! How can I help you today?',
   'Hi there! What can I do for you?',
   'Hey! What do you need help with?']},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye'],
  'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
 {'tag': 'thanks',
  'patterns': ['Thanks',
   'Thank you',
   "That's helpful",
   'Thanks for the help',
   'ok thanks'],
  'responses': ["You're welcome, sir. If you have any other questions or concerns, feel free to reach out. Have a great day!",
   'Happy to help!',
   'Any time!',
   'My pleasure',
   "You're most welcome!"]},
 {'tag': 'about',
  'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
  'responses': ['I am Shifra, your bot assistant',
   "I'm Shifra, an Artificial Intelligent bot"]},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should I call you',
   'whats your name?'],
  'respo